In [7]:
using JuMP
using Gurobi
using AxisArrays

INFO: Recompiling stale cache file /home/manuelli/.julia/lib/v0.5/AxisArrays.ji for module AxisArrays.


In [8]:
# type to store the info about a ContactPoint
type ContactPoint
    name::String
    rot_matrix::Array{Float64,2} # maybe should be some sort of RotationMatrix type
end

# returns the square of the L2 norm of a vector
function l2_norm(x)
    return sum(x.^2)
end

function difference_convex_functions_decomposition(l, force)
    # l = p - r, ray from com to contact location
    
    # the commas are important here to have size (2,) rather
    # than (1,2)
    a = [-l[3],l[2]]
    b = [l[3],-l[1]]
    c = [-l[2],-l[1]]
    d = force[2:3]
    e = force[[1,3]]
    f = force[1:2]

    # this is the decomposition of the cross product into difference
    # of convex functions
    # these are 3-vectors
    p = 1/4 * [l2_norm(a+d),l2_norm(b+e),l2_norm(c+f)]
    q = 1/4 * [l2_norm(a-d),l2_norm(b-e),l2_norm(c-f)]
    
    return p,q
end

# a more convenient method of using JuMP Variables than JuMP Arrays
macro axis_variables(m, var, axes...)
    for arg in axes
        @assert arg.head == :kw "Axis arguments must be of the form `name=domain`"
    end
    names = [arg.args[1] for arg in axes]
    domains = [arg.args[2] for arg in axes]
    ranges = [:(1:length($domain)) for domain in domains]
    axis_args = [Expr(:call, Expr(:curly, :Axis, Expr(:quote, names[i])), domains[i]) for i in eachindex(axes)]
    quote
        vars = @variable $m $var[$(ranges...)]
        $(esc(var)) = $(Expr(:call, :AxisArray, :vars, axis_args...))
    end
end

@axis_variables (macro with 1 method)

In [9]:
contact_names = ["left_foot", "right_foot"]
num_contacts = length(contact_names)


contact_name_to_idx = Dict()
for (idx, name) in enumerate(contact_names)
    contact_name_to_idx[name] = idx
end

# initialize constants
num_timesteps = 5
num_knot_points = num_timesteps + 1
robot_mass = 1
dt = 0.1
gravity = [0,0,-9.8]
dimension = 3

#setup foot positions in world
contact_location = Dict()
contact_location["left_foot"] = [-0.5,0,0]
contact_location["right_foot"] = [0.5,0,0]
mu = 0.5


# com_data
com_data = Dict();
com_data["initial_position"] = [0,0,1]
com_data["final_position"] = [0,0,1]


# weights
weights = Dict()
weights["com_final_position"] = 100
weights["lin_momentum"] = 0.1
weights["forces"] = 0.01
weights["convex_bounds"] = 1e-4 # if this is too large forces will just be zero

# initialize the model
model = Model()


Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is default solver

In [10]:
# add variables to model

# com position
@variable(model, com_position[1:3, 1:num_knot_points])
# @axis_variables(model, com_position, dim=1:3, time=1:num_knot_points)

# linear momentum
@variable(model, lin_momentum[1:3, 1:num_knot_points])

# angular momentum
@variable(model, ang_momentum[1:3, 1:num_knot_points])

# linear momentum dot
@variable(model, lin_momentum_dot[1:3, 1:num_knot_points])

# angular momentum dot
@variable(model, ang_momentum_dot[1:3,1:num_knot_points])

# forces
contact_list = ["left_foot", "right_foot"]
@variable(model, forces[1:3, 1:num_timesteps, 1:num_contacts] >= 0)

# will eventually need something like frames attached to those bodies so we can figure
# the friction cone and stuff
# contact_frames would be an array of rotation matrices basically that
# transform body to world


# torques
# bound them at zero for now
@variable(model, 0 <= torques[1:3, 1:num_timesteps, 1:num_contacts] <= 0)

# bounding variables for l x f terms
@variable(model, l_cross_f_plus[1:dimension, 1:num_timesteps, 1:num_contacts] >= 0)
@variable(model, l_cross_f_minus[1:dimension, 1:num_timesteps, 1:num_contacts] >= 0)

3×5×2 Array{JuMP.Variable,3}:
[:, :, 1] =
 l_cross_f_minus[1,1,1]  l_cross_f_minus[1,2,1]  …  l_cross_f_minus[1,5,1]
 l_cross_f_minus[2,1,1]  l_cross_f_minus[2,2,1]     l_cross_f_minus[2,5,1]
 l_cross_f_minus[3,1,1]  l_cross_f_minus[3,2,1]     l_cross_f_minus[3,5,1]

[:, :, 2] =
 l_cross_f_minus[1,1,2]  l_cross_f_minus[1,2,2]  …  l_cross_f_minus[1,5,2]
 l_cross_f_minus[2,1,2]  l_cross_f_minus[2,2,2]     l_cross_f_minus[2,5,2]
 l_cross_f_minus[3,1,2]  l_cross_f_minus[3,2,2]     l_cross_f_minus[3,5,2]

In [12]:
# add constraints to model
for i=1:num_timesteps
    
    # com position integration constraint
    @constraint(model, com_position[:,i+1] .- (com_position[:,i] .+
        dt/robot_mass.*lin_momentum[:,i]) .== 0)
    
    # linear momentum integration constraint
    @constraint(model, lin_momentum[:,i+1] .- (lin_momentum[:,i] 
            + lin_momentum_dot[:,i]*dt) .== 0)       
    
    # angular momentum integration constraint
    @constraint(model, ang_momentum[:, i+1] .- (ang_momentum[:,i]
            .+ ang_momentum_dot[:,i] * dt) .== 0)
    
    
    total_force = zero(AffExpr) # total force
    total_torque = zero(AffExpr) # total torque
    
    for contact_idx =1:num_contacts
        total_force += forces[:,i,contact_idx]
        total_torque += torques[:,i,contact_idx]
    end
        
    # F = ma
    @constraint(model, lin_momentum_dot[:,i] - (robot_mass*gravity .+ total_force) .== 0)
    
    # the total l x f term
    l_cross_f = zero(AffExpr)
    for contact_idx =1:num_contacts
        
        # lookup string name of contact
        contact_name = contact_names[contact_idx]
        
        # vector from com --> contact location
        l = contact_location[contact_name] .- com_position[:,i]
        force_local = forces[:,i,contact_idx]
        
        # decompose into difference of convex functions
        p,q = difference_convex_functions_decomposition(l, force_local)
        
        # add constraint for convex relaxation
        @constraint(model, p .<= l_cross_f_plus[:,i,contact_idx])
        @constraint(model, q .<= l_cross_f_minus[:,i,contact_idx])
        
        # add the contribution of this contact to the overally l x f term.
        l_cross_f += l_cross_f_plus[:,i,contact_idx] - l_cross_f_minus[:,i,contact_idx]
    end
    
    # angular momentum dot constraint
    @constraint(model, ang_momentum_dot[:,i] .- (l_cross_f .+ total_torque) .== 0)
end

# initial condition constraints
@constraint(model, com_position[:,1] .== com_data["initial_position"])
@constraint(model, lin_momentum[:,1] .== zeros(3))
@constraint(model, lin_momentum_dot[:,1] .== zeros(3))

LoadError: MethodError: no method matching +(::JuMP.GenericAffExpr{Float64,JuMP.Variable}, ::AxisArrays.AxisArray{JuMP.GenericAffExpr{Float64,JuMP.Variable},1,Array{JuMP.GenericAffExpr{Float64,JuMP.Variable},1},Tuple{AxisArrays.Axis{:dim,UnitRange{Int64}}}})[0m
Closest candidates are:
  +(::Any, ::Any, [1m[31m::Any[0m, [1m[31m::Any...[0m) at operators.jl:138
  +(::JuMP.GenericAffExpr{Float64,JuMP.Variable}, [1m[31m::JuMP.GenericQuadExpr{Float64,JuMP.Variable}[0m) at /home/manuelli/.julia/v0.5/JuMP/src/operators.jl:194
  +{R,S}([1m[31m::AbstractArray{R,N}[0m, ::AbstractArray{S,N}) at arraymath.jl:49
  ...[0m

In [50]:
# add objective
com_final_position_cost = weights["com_final_position"]*
l2_norm(com_position[:,end] - com_data["final_position"])

lin_momentum_cost = weights["lin_momentum"]*l2_norm(lin_momentum[:,:,:])    

force_cost = zero(QuadExpr)
convex_bound_cost = zero(QuadExpr)
for contact_idx=1:num_contacts
    force_cost += weights["forces"]*l2_norm(forces[:,:,contact_idx])
    convex_bound_cost += weights["convex_bounds"]*l2_norm(l_cross_f_plus[:,:,contact_idx])
    convex_bound_cost += weights["convex_bounds"]*l2_norm(l_cross_f_minus[:,:,contact_idx])
end

total_objective = com_final_position_cost + lin_momentum_cost
+ force_cost

total_objective += convex_bound_cost

@objective(model, Min, total_objective)

:Min

In [51]:
# solve the model
status = solve(model)

Optimize a model with 84 rows, 210 columns and 294 nonzeros
Model has 81 quadratic objective terms
Model has 60 quadratic constraints
Presolve removed 62 rows and 103 columns
Presolve time: 0.00s
Presolved: 318 rows, 311 columns, 686 nonzeros
Presolved model has 57 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 Dense cols : 1
 Free vars  : 6
 AA' NZ     : 1.863e+03
 Factor NZ  : 4.676e+03
 Factor Ops : 8.779e+04 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.39338246e+01  2.39338246e+01  6.32e+01 3.77e+02  4.00e+01     0s
   1  -1.11400433e+03 -7.39676188e+01  3.31e+01 1.44e+02  1.94e+01     0s
   2  -4.86918065e+01 -7.95131962e+01  5.67e+00 1.07e+02  5.70e+00     0s
   3  -2.65567713e+02 -8.30040663e+01  3.11e+00 9.93e+01  4.34e+00     0s
   4  -1.75891716e+02 -1.11548211e+02  3.00e+00 4.51e+01  2.76e+00     0s
   5  -1.5

:Optimal

In [52]:
# parse solution data
# trailing underscore indicates value of those elements
com_position_ = getvalue(com_position)
lin_momentum_ = getvalue(lin_momentum)
ang_momentum_ = getvalue(ang_momentum)
lin_momentum_dot_ = getvalue(lin_momentum_dot)
ang_momentum_dot_ = getvalue(ang_momentum_dot)
l_cross_f_plus_ = getvalue(l_cross_f_plus)
l_cross_f_minus_ = getvalue(l_cross_f_minus)
forces_ = getvalue(forces)

p_ = zeros(l_cross_f_plus_)
q_ = zeros(l_cross_f_plus_)

# populate q_, v_ check tightness with l_cross_f_plus/minus
for i=1:num_timesteps
    for contact_idx = 1:num_contacts
        contact_name = contact_names[contact_idx]
        l = contact_location[contact_name] - com_position_[:,i]
        force_local = forces_[:,i,contact_idx]
        p_[:,i,contact_idx], q_[:,i,contact_idx] = 
        difference_convex_functions_decomposition(l,force_local)
    end
end

In [53]:
p_slack = l_cross_f_plus_ - p_
display(p_slack)

3×5×2 Array{Float64,3}:
[:, :, 1] =
 1.44552e-5  1.96132e-5  8.36794e-6  1.18531e-5  1.23804e-7
 9.87983e-6  1.28081e-5  9.00727e-5  9.33376e-6  0.00119872
 0.0278221   0.0054672   0.00555865  0.00459316  0.00676697

[:, :, 2] =
 1.44552e-5  3.3243e-5   1.02678e-5  1.11501e-5  4.06503e-5
 2.90524e-5  4.12641e-5  7.63412e-6  1.23212e-5  8.54307e-5
 0.0278221   0.0054423   0.00554111  0.00450069  0.00810774

In [45]:
p = zeros(l_cross_f_minus_)
size(p)

(3,5,2)